In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.2
3.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('num_events', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.1_Dike Investment Costs')
ArrayOutcome('A.1_Expected Number of Deaths')
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.2_Dike Investment Costs')
ArrayOutcome('A.2_Expected Number of Deaths')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.3_Dike Investment Costs')
ArrayOutcome('A.3_Expected Number of Deaths')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.4_Dike Investment Costs')
ArrayOutcome('A.4_Expected Number of Deaths')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('A.5_Dike Investment Costs')
ArrayOutcome('A.5_Expected Number of Deaths')
ArrayOutcome('RfR Total Costs')
ArrayOutcome('Expected Evacuation Costs')


In [8]:
outcomes= [i for i in dike_model.outcomes]
outcomes

[ArrayOutcome('A.1_Expected Annual Damage'),
 ArrayOutcome('A.1_Dike Investment Costs'),
 ArrayOutcome('A.1_Expected Number of Deaths'),
 ArrayOutcome('A.2_Expected Annual Damage'),
 ArrayOutcome('A.2_Dike Investment Costs'),
 ArrayOutcome('A.2_Expected Number of Deaths'),
 ArrayOutcome('A.3_Expected Annual Damage'),
 ArrayOutcome('A.3_Dike Investment Costs'),
 ArrayOutcome('A.3_Expected Number of Deaths'),
 ArrayOutcome('A.4_Expected Annual Damage'),
 ArrayOutcome('A.4_Dike Investment Costs'),
 ArrayOutcome('A.4_Expected Number of Deaths'),
 ArrayOutcome('A.5_Expected Annual Damage'),
 ArrayOutcome('A.5_Dike Investment Costs'),
 ArrayOutcome('A.5_Expected Number of Deaths'),
 ArrayOutcome('RfR Total Costs'),
 ArrayOutcome('Expected Evacuation Costs')]

## Emma, please run up to the cell below and let me know if the percentage increases on your machine.

In [9]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [03:31<00:00,  1.06s/it]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


Sobol

In [16]:
from SALib.analyze import sobol
from ema_workbench import Samplers
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

In [17]:
def analyze(results, kpi,planning_step):
    """analyze results using SALib sobol, returns a dataframe"""
    _, outcomes = results
    
    problem = get_SALib_problem(dike_model.uncertainties)
    y = outcomes[kpi][planning_step]
    sobol_indices = sobol.analyze(problem, y, calc_second_order=True, print_to_console=False)
    sobol_stats = {key: sobol_indices[key] for key in ["ST", "ST_conf", "S1", "S1_conf"]}
    sobol_stats = pd.DataFrame(sobol_stats, index=problem["names"])
    sobol_stats.sort_values(by="ST", ascending=False)
    s2 = pd.DataFrame(sobol_indices["S2"], index=problem["names"], columns=problem["names"])
    s2_conf = pd.DataFrame(sobol_indices["S2_conf"], index=problem["names"], columns=problem["names"])
    return sobol_stats, s2, s2_conf

In [ ]:
n_scenarios=10  #100 #1000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results=evaluator.perform_experiments(scenarios=n_scenarios,uncertainty_sampling=Samplers.SOBOL) #
 
# Save the results
save_results(results, "dike_model_test_sobol_no_policies.tar.gz")

end of Sobol section

In [10]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage', 'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths', 'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs', 'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage', 'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths', 'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,79,337.276931,1.0,0.758896,168.441446,1.5,0.791519,279.437359,1.0,0.091196,...,1,10,1,10,0,2,0,4,0,dikesnet
1,70,65.773643,1.5,0.183920,231.059721,10.0,0.138336,52.542750,1.5,0.734823,...,1,10,1,10,0,2,0,5,0,dikesnet
2,94,56.042454,1.0,0.127433,271.365404,10.0,0.566181,79.621933,1.5,0.309901,...,1,10,1,10,0,2,0,6,0,dikesnet
3,77,289.524991,1.5,0.363640,99.066977,1.0,0.817029,157.255506,1.0,0.333062,...,1,10,1,10,0,2,0,7,0,dikesnet
4,118,330.144524,10.0,0.697123,292.847364,1.0,0.296120,223.950628,10.0,0.384217,...,1,10,1,10,0,2,0,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,98,47.638562,1.0,0.960581,325.738316,1.5,0.447279,109.815867,10.0,0.422412,...,5,3,8,7,7,9,3,49,3,dikesnet
196,79,88.150139,10.0,0.673075,86.614612,10.0,0.889908,281.594048,10.0,0.272361,...,5,3,8,7,7,9,3,50,3,dikesnet
197,37,316.067894,10.0,0.270241,262.047436,1.0,0.061784,161.678654,1.0,0.748654,...,5,3,8,7,7,9,3,51,3,dikesnet
198,83,256.604995,1.0,0.410166,225.406549,1.5,0.527519,166.241749,1.0,0.621161,...,5,3,8,7,7,9,3,52,3,dikesnet


In [15]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes['RfR Total Costs'])

,0,1,2
0,473900000.0,473900000.0,302400000.0
1,473900000.0,473900000.0,302400000.0
2,473900000.0,473900000.0,302400000.0
3,473900000.0,473900000.0,302400000.0
4,473900000.0,473900000.0,302400000.0
...,...,...,...
195,377300000.0,504600000.0,589200000.0
196,377300000.0,504600000.0,589200000.0
197,377300000.0,504600000.0,589200000.0
198,377300000.0,504600000.0,589200000.0


In [12]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [13]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 10 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:12<00:00, 24.96it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [14]:
experiments, outcomes = results

In [15]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,1.456509e+08,0.140958,1.140604e+09,2.037104,6.270157e+06,0.002970,3.532720e+08,0.317249,253800000.0,0.0
1,5.397251e+07,0.000000,9.280847e+07,0.078009,2.169010e+07,0.034686,1.016735e+06,0.000461,0.000000e+00,0.000000,253800000.0,0.0
2,5.397251e+07,0.000000,1.054994e+08,0.093604,0.000000e+00,0.000000,1.517131e+07,0.006695,8.875536e+08,0.728919,253800000.0,0.0
3,5.397251e+07,0.000000,2.988322e+08,0.370025,8.400418e+08,1.974631,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,4.766690e+06,0.006138,6.334425e+08,1.421049,1.302479e+07,0.007597,0.000000e+00,0.000000,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,6.449472e+08,0.424723,0.000000e+00,0.000000,2.879840e+07,0.000000,9.763206e+07,0.042576,0.000000e+00,0.000000,369700000.0,0.0
296,0.000000e+00,0.000000,1.832255e+08,0.164428,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
297,1.118402e+08,0.064348,8.564205e+08,0.644776,2.879840e+07,0.000000,7.329400e+07,0.027965,0.000000e+00,0.000000,369700000.0,0.0
298,3.704246e+08,0.324836,2.042977e+08,0.248160,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
